<a href="https://colab.research.google.com/github/arockiasachin/ContextualFinAi/blob/main/Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install  qdrant-client fastembed langchain_openai langchain_qdrant langchain-community lark google-generativeai langchain_google_genai langchain_cohere ragas langchain-google-vertexai

  Using cached langchain_google_vertexai-2.0.9-py3-none-any.whl.metadata (3.8 kB)
  Using cached google_cloud_aiplatform-1.75.0-py2.py3-none-any.whl.metadata (31 kB)
  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached google_cloud_bigquery-3.27.0-py2.py3-none-any.whl.metadata (8.6 kB)
  Using cached google_cloud_resource_manager-1.14.0-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached shapely-2.0.6-cp312-cp312-win_amd64.whl.metadata (7.2 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_crc32c-1.6.0-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached grpc_google_iam_v1-0.13.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached langchain_google_vertexai-2.0.9-py3-none-any.whl (90 kB)
   ----------------------------------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\SachinSlade\\anaconda3\\envs\\Unstructured\\Lib\\site-packages\\shapely\\__init__.py'
Consider using the `--user` option or check the permissions.



In [ ]:
import os
import json
import numpy as np
from uuid import uuid4
import getpass
import pandas as pd
# --- Qdrant Client and Related Models ---
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    VectorParams,
    SparseVectorParams,
    SparseIndexParams,
    PointStruct,
    Filter,
    FieldCondition,
    Range,
)

# --- OpenAI ---
import openai
from openai import OpenAI

# --- Langchain ---
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import QdrantVectorStore, RetrievalMode, FastEmbedSparse
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

# --- Embeddings and Reranking ---
from langchain_core.embeddings import Embeddings
from fastembed import SparseTextEmbedding
from langchain_cohere import CohereRerank


# --- Google Generative AI ---
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI


#---------Ragas----------
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from datasets import Dataset

# --- Other ---
from collections.abc import MutableMapping


In [ ]:
os.environ["OPENAI_API_KEY"] = "API KEY HERE"
os.environ["GOOGLE_API_KEY"] = "API KEY HERE"
os.environ["COHERE_API_KEY"] = "API KEY HERE"

In [ ]:
qdrant_client = QdrantClient(
    url="https://d0e39225-965c-4bfe-aa0d-6cb22660e552.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="API KEY HERE",
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='Amazon'), CollectionDescription(name='amazon_reports'), CollectionDescription(name='terraforming')]


# Data Loading

In [ ]:
# Step 2: Load the data file with dense embeddings
with open(file_path, "r") as f:
    data = json.load(f)

# Flatten nested metadata
def flatten_dict(d, parent_key="", sep="_"):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Extract content, embeddings, and metadata
texts = [doc["page_content"] for doc in data]
dense_embeddings = [doc["embeddings"] for doc in data]
metadata = [flatten_dict(doc["metadata"]) for doc in data]

In [ ]:
texts[0]

In [ ]:
# Initialize the SPLADE model
model_name = "prithivida/Splade_PP_en_v1"
sparse_model = SparseTextEmbedding(model_name=model_name)

In [ ]:
# Generate sparse embeddings
sparse_embeddings = list(sparse_model.embed(texts))
sparse_vectors[0]

In [ ]:
# Paths to the input and output JSON files
INPUT_FILE = file_path # Replace with your file path
OUTPUT_FILE = r"D:\unstructured\FinalNew52\langchain_documents_with_hybrid_embeddings.json"

# Step 1: Load the existing JSON data
with open(INPUT_FILE, "r") as file:
    json_data = json.load(file)

# Step 2: Add sparse embeddings to each document
if len(json_data) != len(sparse_embeddings):
    raise ValueError("The number of sparse embeddings must match the number of JSON entries.")

for i, doc in enumerate(json_data):
    # Convert SparseEmbedding object to JSON-compatible format
    sparse_embedding = {
        "values": sparse_embeddings[i].values.tolist(),
        "indices": sparse_embeddings[i].indices.tolist()
    }
    # Add to the document
    doc["sparse_embeddings"] = sparse_embedding

# Step 3: Save the updated JSON data
with open(OUTPUT_FILE, "w") as file:
    json.dump(json_data, file, indent=4)

print(f"Sparse embeddings have been added and saved to {OUTPUT_FILE}")

Sparse embeddings have been added and saved to D:\unstructured\FinalNew52\langchain_documents_with_hybrid_embeddings.json


In [ ]:
SparseTextEmbedding.list_supported_models()

[{'model': 'prithivida/Splade_PP_en_v1',
  'vocab_size': 30522,
  'description': 'Independent Implementation of SPLADE++ Model for English.',
  'license': 'apache-2.0',
  'size_in_GB': 0.532,
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1'},
  'model_file': 'model.onnx'},
 {'model': 'prithvida/Splade_PP_en_v1',
  'vocab_size': 30522,
  'description': 'Independent Implementation of SPLADE++ Model for English.',
  'license': 'apache-2.0',
  'size_in_GB': 0.532,
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1'},
  'model_file': 'model.onnx'},
 {'model': 'Qdrant/bm42-all-minilm-l6-v2-attentions',
  'vocab_size': 30522,
  'description': 'Light sparse embedding model, which assigns an importance score to each token in the text',
  'license': 'apache-2.0',
  'size_in_GB': 0.09,
  'sources': {'hf': 'Qdrant/all_miniLM_L6_v2_with_attentions'},
  'model_file': 'model.onnx',
  'additional_files': ['stopwords.txt'],
  'requires_idf': True},
 {'model': 'Qdrant/bm25',
  'description': 'BM25 as sparse embedd

# Qdrant Setup

In [ ]:
qdrant_client.create_collection(
    collection_name="amazon_reports",
    vectors_config={
        "text-dense": VectorParams(
            size=len(dense_embeddings[0]),  # Dimension of dense vectors
            distance=Distance.COSINE
        )
    },
    sparse_vectors_config={
        "text-sparse": SparseVectorParams(
            index=SparseIndexParams(on_disk=False)  # Configure sparse index
        )
    }
)

True

In [ ]:
# Step 6: Prepare data points with both dense and sparse vectors
print("Preparing data points...")
points = []
for idx, sparse_vector in enumerate(sparse_embeddings):
    # Merge metadata with text content
    payload = metadata[idx].copy()  # Copy metadata
    payload["text"] = texts[idx]  # Add the document text to the payload

    points.append(
        PointStruct(
            id=idx,
            vector={
                "text-dense": dense_embeddings[idx],  # Dense vector
                "text-sparse": {"indices": sparse_vector.indices, "values": sparse_vector.values}  # Sparse vector
            },
            payload=payload  # Include both metadata and text
        )
    )

In [ ]:
batch_size = 50  # Adjust based on your data size
for i in range(0, len(points), batch_size):
    batch = points[i:i+batch_size]
    qdrant_client.upsert(collection_name="amazon_reports", points=batch)

In [ ]:
# Initialize QdrantVectorStore
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="amazon_reports",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="text-dense",  # Replace with the correct dense vector name
    sparse_vector_name="text-sparse",
    content_payload_key="text",




)

In [ ]:
results = vector_store.similarity_search_with_score(query=query_text, k=3)


In [ ]:
# Display Results
print("\n--- Hybrid Retrieval Results ---")
for doc, score in results:
    print(f"[Score: {score:.4f}] Content: {doc.page_content} | Metadata: {doc.metadata}")


--- Hybrid Retrieval Results ---
[Score: 0.5000] Content: AWS has an $85B annualized revenue run rate, is still early in its adoption curve, but at a juncture where it’s critical to stay focused on what matters most to customers over the long-haul. Despite growing 29% year-over- year (“YoY”) in 2022 on a $62B revenue base, AWS faces short-term headwinds right now as companies are being more cautious in spending given the challenging, current macroeconomic conditions. While some companies might obsess over how they could extract as much money from customers as possible in these tight times, it’s neither what customers want nor best for customers in the long term, so we’re taking a different tack. One of the many advantages of AWS and cloud computing is that when your business grows, you can seamlessly scale up; and conversely, if your business contracts, you can choose to give us back that capacity and cease paying for it. This elasticity is unique to the cloud, and doesn’t exist when 

# Querying the DB

In [ ]:
def get_dense_embedding(text):
    # Create an embedding using the new OpenAI API method
    response = openai.embeddings.create(input=text, model="text-embedding-3-large")
    return response.data[0].embedding

# Sparse Embedding Model (SPLADE via FastEmbed)
sparse_model = SparseTextEmbedding(model_name="prithivida/Splade_PP_en_v1")

# Step 2: Query Text Input
query_text = "Amazon AWS revenue growth in 2023"

# Step 3: Generate Dense and Sparse Embeddings
print("Generating dense and sparse embeddings...")
query_dense_vector = get_dense_embedding(query_text)
sparse_embedding = list(sparse_model.embed([query_text]))[0]

query_sparse_vector = {
    "indices": sparse_embedding.indices,
    "values": sparse_embedding.values
}
# Compute sparse and dense vectors
query_indices = query_sparse_vector['indices']
query_values = query_sparse_vector['values']
query_dense_vector = query_dense_vector

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating dense and sparse embeddings...


In [ ]:
query_sparse_vector['values']

array([0.21064357, 0.54518491, 0.98074651, 0.17624055, 0.36889499,
       0.82288623, 0.59506387, 1.27148747, 0.28183684, 1.0800854 ,
       1.91824389, 0.92381424, 0.06167603, 0.43334585, 0.12849194,
       0.2201781 , 0.04837064, 1.19580293, 1.11256003, 0.05031812,
       1.92592824, 2.56553984, 2.00859976, 2.03181243])

In [ ]:
query_indices

'indices'

In [ ]:
search_results = qdrant_client.query_points(
    collection_name="amazon_reports",
    prefetch=[
        models.Prefetch(
            query=models.SparseVector(indices=query_indices, values=query_values),
            using="text-sparse",
            limit=20,
        ),
        models.Prefetch(
            query=query_dense_vector,
            using="text-dense",
            limit=20,
        ),
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),  # Use Reciprocal Rank Fusion
    limit=10,  # Retrieve top 10 results
)

In [ ]:
search_results

QueryResponse(points=[ScoredPoint(id=107, version=2, score=0.5769231, payload={'element_id': '844a9b594fe796512b3a867c8eefe27a', 'filename': 'Amazon-com-Inc-2023-Annual-Report.pdf', 'page_number': 36, 'is_continuation': False, 'text': 'International sales increased 11% in 2023, compared to the prior year. The sales growth primarily reflects increased unit sales, primarily by third-party sellers, advertising sales, and subscription services. Increased unit sales were driven largely by our continued focus on price, selection, and convenience for our customers, including from our shipping offers. Changes in foreign exchange rates increased International net sales by $88 million in 2023.\n\n24\n\nAWS sales increased 13% in 2023, compared to the prior year. The sales growth primarily reflects increased customer usage, partially offset by pricing changes, primarily driven by long-term customer contracts.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=415, version=8, score=

In [ ]:
# Process Qdrant results to extract payload content
processed_results = [
    {
        "filename": res.payload['filename'],
        "page_number": res.payload['page_number'],
        "text": res.payload['text']
    }
    for res in search_results.points  # Iterate over points in search_results
]

In [ ]:
processed_results

[{'filename': 'Amazon-com-Inc-2023-Annual-Report.pdf',
  'page_number': 36,
  'text': 'International sales increased 11% in 2023, compared to the prior year. The sales growth primarily reflects increased unit sales, primarily by third-party sellers, advertising sales, and subscription services. Increased unit sales were driven largely by our continued focus on price, selection, and convenience for our customers, including from our shipping offers. Changes in foreign exchange rates increased International net sales by $88 million in 2023.\n\n24\n\nAWS sales increased 13% in 2023, compared to the prior year. The sales growth primarily reflects increased customer usage, partially offset by pricing changes, primarily driven by long-term customer contracts.'},
 {'filename': 'Amazon-com-Inc-2023-Shareholder-Letter.pdf',
  'page_number': 1,
  'text': 'amazon\n\nDear Shareholders:\n\nLast year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The r

In [ ]:
# Step 1: Format search results into context for OpenAI
context = "\n\n".join(
    [f"Source: {res['filename']} (Page {res['page_number']}): {res['text']}\n\n\n" for res in processed_results]
)

# Step 2: Define the query and prompt
query = "Summarize the revenue growth of Amazon AWS in 2023 and the contributing factors."
prompt = f"Context:\n{context}\n\nQuery: {query}. Also mention every source filename and page number like a proper refernce \n Template: \n Answer: \n Source:"

In [ ]:
prompt

'Context:\nSource: Amazon-com-Inc-2023-Annual-Report.pdf (Page 36): International sales increased 11% in 2023, compared to the prior year. The sales growth primarily reflects increased unit sales, primarily by third-party sellers, advertising sales, and subscription services. Increased unit sales were driven largely by our continued focus on price, selection, and convenience for our customers, including from our shipping offers. Changes in foreign exchange rates increased International net sales by $88 million in 2023.\n\n24\n\nAWS sales increased 13% in 2023, compared to the prior year. The sales growth primarily reflects increased customer usage, partially offset by pricing changes, primarily driven by long-term customer contracts.\n\n\n\n\nSource: Amazon-com-Inc-2023-Shareholder-Letter.pdf (Page 1): amazon\n\nDear Shareholders:\n\nLast year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reasons are many, but start with the progres

In [ ]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

response = client.chat.completions.create(
    model="gpt-4o-mini",  # Replace with your desired model
    messages=[
        {"role": "system", "content": "You are a helpful assistant summarizing financial data."},
        {"role": "user", "content": prompt}
    ]
)

# Output the result
print("Generated Response:")
print(response.choices[0].message.content.strip())


Generated Response:
Answer: In 2023, Amazon Web Services (AWS) experienced a revenue growth of 13% compared to the prior year, reaching $91 billion, up from $80 billion in 2022. This growth was primarily driven by increased customer usage, although it was partially offset by pricing changes linked to long-term customer contracts. Notably, AWS focused on helping customers optimize their cloud usage and reduce costs during uncertain economic conditions, leading to a more efficient use of resources. By the end of 2023, there was a noticeable increase in new deals and larger customer commitments, indicating a positive outlook for continued growth in the AWS segment.

Sources:
- Amazon-com-Inc-2023-Annual-Report.pdf, Page 36
- Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 1
- Amazon-2022-Annual-Report.pdf, Page 4


# Langchain

In [ ]:
with open(file_path, "r") as f:
    data = json.load(f)

# Step 2: Function to Flatten Nested Metadata
def flatten_dict(d, parent_key="", sep="_"):
    """Recursively flattens a nested dictionary."""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Step 3: Extract `page_content` and Flattened Metadata
texts = [doc["page_content"] for doc in data]
metadata = [flatten_dict(doc["metadata"]) for doc in data]

# Step 4: Create LangChain Document Objects
documents = [
    Document(page_content=text, metadata=meta) for text, meta in zip(texts, metadata)
]


# Generate unique IDs for the documents
uuids = [str(uuid4()) for _ in range(len(documents))]

# Verify the Documents
for i, doc in enumerate(documents[:3]):  # Print first 3 documents
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}\n")


Document 1:
Content: 2

0

3

amazon

A N N U A L R E P O R T

amazon

Dear Shareholders:

Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reasons are many, but start with the progress we’ve made in our financial results and customer experiences, and extend to our continued innovation and the remarkable opportunities in front of us.

In 2023, Amazon’s total revenue grew 12% year-over-year (“YoY”) from $514B to $575B. By segment, North America revenue increased 12% YoY from $316B to $353B, International revenue grew 11% YoY from $118B to $131B, and AWS revenue increased 13% YoY from $80B to $91B.

Further, Amazon’s operating income and Free Cash Flow (“FCF”) dramatically improved. Operating income in 2023 improved 201% YoY from $12.2B (an operating margin of 2.4%) to $36.9B (an operating margin of 6.4%). Trailing Twelve Month FCF adjusted for equipment finance leases improved from -$12.8B in 2022 to $35.5B (up $48.3B).

While

In [ ]:
# Step 2: Initialize Dense Embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Step 3: Initialize Sparse Embeddings
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
qdrant_client.create_collection(
    collection_name="Amazon",
    vectors_config={
        "dense": VectorParams(size=3072, distance=Distance.COSINE)  # Dense vector name is "default"
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams(index=SparseIndexParams(on_disk=False))  # Sparse vector name
    }
)


In [ ]:
# Recreate the Collection with Default Named Vectors

# Initialize QdrantVectorStore
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="Amazon",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",  # Replace with the correct dense vector name
    sparse_vector_name="sparse"


)

print("QdrantVectorStore initialized successfully!")

QdrantVectorStore initialized successfully!


In [ ]:
# Step 7: Add Documents to Qdrant
vector_store.add_documents(documents=documents, ids=uuids)

print("Documents inserted successfully!")


Documents inserted successfully!


In [ ]:
# Query for Hybrid Retrieval
query_text = "What is the AWS revenue for 2023?"
results = vector_store.similarity_search_with_score(query=query_text, k=3)

# Display Results
print("\n--- Hybrid Retrieval Results ---")
for doc, score in results:
    print(f"[Score: {score:.4f}] Content: {doc.page_content} | Metadata: {doc.metadata}")



--- Hybrid Retrieval Results ---
[Score: 0.5000] Content: Shifting to AWS, we started 2023 seeing substantial cost optimization, with most companies trying to save money in an uncertain economy. Much of this optimization was catalyzed by AWS helping customers use the cloud more efficiently and leverage more powerful, price-performant AWS capabilities like Graviton chips (our generalized CPU chips that provide ~40% better price-performance than other leading x86 processors), S3 Intelligent Tiering (a storage class that uses AI to detect objects accessed less frequently and store them in less expensive storage layers), and Savings Plans (which give customers lower prices in exchange for longer commitments). This work diminished short-term revenue, but was best for customers, much appreciated, and should bode well for customers and AWS longer-term. By the end of 2023, we saw cost optimization attenuating, new deals accelerating, customers renewing at larger commitments over longer time p

# Self Querying Retrieval

In [ ]:
# Initialize the Gemini Model
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

In [ ]:
# Step 2: Initialize Dense Embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Step 3: Initialize Sparse Embeddings
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")
# Initialize QdrantVectorStore
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="Amazon",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",  # Replace with the correct dense vector name
    sparse_vector_name="sparse"
)

print("QdrantVectorStore initialized successfully!")

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

QdrantVectorStore initialized successfully!


In [ ]:
# Query using available metadata fields
print(compression_retriever.invoke("What is the AWS role in the Amazon annual report?"))

[Document(metadata={'element_id': '735075edbf527c6300fee3d53dd25371', 'filename': 'Amazon-com-Inc-2023-Annual-Report.pdf', 'page_number': 3, 'is_continuation': False, '_id': 'f6a622dd-1a1a-4dde-8426-41dc56b98e58', '_collection_name': 'Amazon', 'relevance_score': 0.14651443}, page_content='Shifting to AWS, we started 2023 seeing substantial cost optimization, with most companies trying to save money in an uncertain economy. Much of this optimization was catalyzed by AWS helping customers use the cloud more efficiently and leverage more powerful, price-performant AWS capabilities like Graviton chips (our generalized CPU chips that provide ~40% better price-performance than other leading x86 processors), S3 Intelligent Tiering (a storage class that uses AI to detect objects accessed less frequently and store them in less expensive storage layers), and Savings Plans (which give customers lower prices in exchange for longer commitments). This work diminished short-term revenue, but was best

In [ ]:

query ="What is the AWS role in the Amazon annual report?"
Cohere_Retriever= compression_retriever.get_relevant_documents(query)
Cohere_Retriever

[Document(metadata={'element_id': '735075edbf527c6300fee3d53dd25371', 'filename': 'Amazon-com-Inc-2023-Annual-Report.pdf', 'page_number': 3, 'is_continuation': False, '_id': 'f6a622dd-1a1a-4dde-8426-41dc56b98e58', '_collection_name': 'Amazon', 'relevance_score': 0.14651443}, page_content='Shifting to AWS, we started 2023 seeing substantial cost optimization, with most companies trying to save money in an uncertain economy. Much of this optimization was catalyzed by AWS helping customers use the cloud more efficiently and leverage more powerful, price-performant AWS capabilities like Graviton chips (our generalized CPU chips that provide ~40% better price-performance than other leading x86 processors), S3 Intelligent Tiering (a storage class that uses AI to detect objects accessed less frequently and store them in less expensive storage layers), and Savings Plans (which give customers lower prices in exchange for longer commitments). This work diminished short-term revenue, but was best

In [ ]:
#filenames = os.listdir("/content/drive/MyDrive/Final52RAG/input")
filenames = os.listdir(f"D:\Amazon")

# Print the filenames
print("Files in the folder:")
for filename in filenames:
    print(filename)



Files in the folder:
2023-political-engagement-statement.pdf
Amazon's-Tax-Principles.pdf
Amazon-2022-Annual-Report.pdf
Amazon-com-Inc-2023-Annual-Report.pdf
Amazon-com-Inc-2023-Shareholder-Letter.pdf
Amazon-com-Inc-2024-Proxy-Statement.pdf
amazon-com-inc-bylaws-may-3-2024-1.pdf
Amazon-s-Global-Economic-Impact-and-Tax-Contribution-2023.pdf
Amazon-Sustainable-Bond-Framework.pdf
amazon_code_of_conduct.pdf
amazon_corporate_governance_guidelines.pdf
amazon_officers.pdf
amzn-2023-conflict-minerals-report.pdf
AMZN-Q1-2024-Earnings-Release.pdf
AMZN-Q2-2024-Earnings-Release.pdf
AMZN-Q3-2024-Earnings-Release.pdf
Guide to 10k.pdf
Note-on-Alignment-with-Paris-Agreement.pdf


<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\SachinSlade\AppData\Local\Temp\ipykernel_3924\3886577819.py:2: SyntaxWarning: invalid escape sequence '\A'
  filenames = os.listdir(f"D:\Amazon")


In [ ]:
ContextualCompressionRetriever??

In [ ]:
# Metadata schema definition
metadata_field_info = [
    AttributeInfo(
        name="element_id",
        description="A unique identifier for the document element",
        type="string",
    ),
    AttributeInfo(
        name="filename",
        description = f"The name of the file where the document is stored. List of filenames are: {', '.join(filenames)}",
        type="string",
    ),
    AttributeInfo(
        name="page_number",
        description="The page number where the document content is located",
        type="integer",
    ),
    AttributeInfo(
        name="is_continuation",
        description="A flag indicating whether the document content continues from a previous page",
        type="boolean",
    ),
]

# Content description
document_content_description = "Text extracted from the Amazon Annual Report, including specific metadata fields like element ID, filename, and page information."


In [ ]:
# Step 3: Define the Prompt Template
prompt_template = """
You are an expert assistant tasked with answering questions based on retrieved documents.
Use the retrieved context to provide accurate, concise, and direct answers to the user's question.
If the information is not available in the context, respond with "I don't have enough information to answer that."

Context:
{context}

Question:
{question}

Final Answer:
[Provide a concise answer to the question here]

Additional Context: [Provide supporting information]
"""

qa_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)



In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm=gemini_llm,
    vectorstore=vector_store,
    search_kwargs={"k": 10},
    document_contents=document_content_description,  # REQUIRED
    metadata_field_info=metadata_field_info,
    verbose=True
)


In [ ]:
print(retriever.invoke(query))

[Document(metadata={'element_id': '3a017cb950d49245eb9f00eff9b88a35', 'filename': 'Amazon-Sustainable-Bond-Framework.pdf', 'page_number': 8, 'is_continuation': False, '_id': 'b90eb226-783f-4e4f-a9c0-6770aec2b4a1', '_collection_name': 'Amazon'}, page_content='25 https://blog.aboutamazon.eu/aws/amazon-web-services-backs-new-swedish-water-conservation-project-to-protect-against-flooding-boost-water-quality-and-biodiversity\n\n26 https://blog.aboutamazon.eu/aws/local-community-buildings-in-ireland-to-be-heated-by-amazon-data-centre\n\n27 https://www.aboutamazon.com/news/amazon-offices/the-next-chapter-for-hq2-sustainable-buildings-surrounded-by-nature\n\n28 https://blog.aboutamazon.eu/sustainability/amazon-supports-fight-against-climate-change-effects-on-european-cities\n\namazon\n\nAmazon Sustainable Bond Framework\n\n8\n\n4.4.2 Impact Reporting\n\nExamples of the impact metrics that may be included in any report we publish may include:\n\nProject Category Example Impact Metrics Clean Tra

In [ ]:
self_query_retriever = RetrievalQA.from_chain_type(
    llm=gemini_llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)
# Initialize the Cohere Reranker with version 3.5
cohere_reranker = CohereRerank(model="rerank-v3.5")

# Use ContextualCompressionRetriever to integrate Cohere Rerank
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_reranker,
    base_retriever=retriever  # Your base retriever setup (e.g., self_query_retriever)
)
Cohere_Retriever= compression_retriever.get_relevant_documents(query)
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.output_parser import StrOutputParser

# Step 1: Define Document Formatter
def format_docs(docs):
    """Format LangChain documents into readable strings."""
    return "\n\n".join(
        f"Filename: {doc.metadata.get('filename', 'Unknown Filename')}\n"
        f"Page Number: {doc.metadata.get('page_number', 'N/A')}\n"
        f"Content: {doc.page_content}"
        for doc in docs
    )

# Step 2: Define Final Answer Extractor
def extract_final_answer(output):
    """Extract the final answer from the model's response."""
    return output.strip()
# Step 4: Create the Retrieval Chain
# Step 5: Create the Retrieval Chain
retrieval_chain = (
    {
        "context": RunnableLambda(lambda _: format_docs(Cohere_Retriever)),  # Use pre-retrieved docs
        "question": RunnablePassthrough(),  # Pass the question directly
    }
    | RunnableLambda(lambda inputs: qa_prompt.format(context=inputs["context"], question=inputs["question"]))  # Format the prompt
    | gemini_llm  # Pass the formatted prompt to the LLM
    | StrOutputParser()  # Parse the output to a string
    #| RunnableLambda(lambda x: extract_final_answer(x))  # Extract final answer
)


In [ ]:
query ="What is the Difference of footage between 2022 and 2023 for Item 2. Properties"
response = retrieval_chain.invoke(query)  # Use 'invoke' for modern LangChain

In [ ]:
response

'Leased square footage increased by 46,608 thousand square feet and owned square footage increased by 18,236 thousand square feet between 2022 and 2023.\n\nAdditional Context:\n2022:  617,483 thousand sq ft (leased) and 43,927 thousand sq ft (owned)\n2023:  664,091 thousand sq ft (leased) and 52,163 thousand sq ft (owned)\nDifference (Leased): 664,091 - 617,483 = 46,608\nDifference (Owned): 52,163 - 43,927 = 18,236'

In [ ]:
# Run queries and print results
for query in queries:
    print(f"Query: {query}")
    response = retrieval_qa_chain.invoke({"query": query})  # Use 'invoke' for modern LangChain
    print(f"Answer: {response['result']}")
    print("-" * 80)

Query: Find projects or products released after 2020 related to renewable energy or clean transportation that have impactful metrics or ratings above 8.5.
Answer: This request can't be fulfilled. While the provided text references Amazon's investments in renewable energy and clean transportation, including specific examples like electric vehicle fleets and charging stations, it doesn't offer specific metrics or ratings for individual projects or products released after 2020.  Therefore, it's not possible to identify projects or products meeting the specified criteria based on the available information.

--------------------------------------------------------------------------------
Query: List all projects in the clean transportation category, such as electric vehicles, charging infrastructure, or alternative delivery systems.
Answer: The following are listed as example projects under the Clean Transportation category:

* Acquisition of electric vehicles for transportation fleets.
* A

In [ ]:
response = retrieval_chain.invoke(query)

In [ ]:
response

"Andy Jassy stated that it was a good start to the year across the business, reflected in both customer experience improvements and financial results. He highlighted the reaccelerated growth rate of AWS, driven by companies renewing infrastructure modernization efforts and the appeal of AWS's AI capabilities; the expansion of selection, lower prices, and faster delivery speeds in the Stores business; and the continued benefit to Advertising efforts from the growth of Stores and Prime Video. He also expressed excitement about future potential to improve customers' lives."